# Variational Quantum Eigensolver (VQE) — Detailed Notes (Session 9)
**Course:** CS490/5590 — Quantum Computing Applications in Data Science, AI, & Deep Learning  
**Instructor:** Luke Miller  

> **Purpose.**  These notes expand the Session-9 slides into a self-contained reference. You will learn how VQE maps chemistry or optimisation Hamiltonians to qubit operators, chooses a parametrised **ansatz**, evaluates the energy on a quantum device, and iteratively minimises it with a classical optimiser.  Mini-exercises are provided at the end.

---

## Session Road-map  

1. Recap: QAOA & variational mindset  
2. VQE fundamentals and variational principle  
3. Building molecular Hamiltonians (second quantisation → Pauli sums)  
4. Ansatz families (hardware-efficient vs UCCSD)  
5. Expectation estimation & measurement grouping  
6. Classical optimisers (SPSA, COBYLA) and hybrid loop  
7. Qiskit Nature workflow (`Molecule → Hamiltonian → VQE`)  
8. Emerging ideas: Stochastic Quantum Dynamics (SQD) & SKQD  
9. AI / DS applications  
10. Q&A  

---

## 0) Recap — QAOA vs. VQE

| Aspect              | QAOA                               | VQE                               |
|---------------------|------------------------------------|-----------------------------------|
| Objective           | **Maximise** cost (Ising)         | **Minimise** energy of Hamiltonian|
| Cost operator       | Problem-specific $H_C$           | Molecular / Ising Hamiltonian $H$|
| Mixer operator      | Standard $H_B$ or custom        | None (ansatz already explores)    |
| Metric              | Approximation ratio                | Energy (upper bound on $E_0$)   |
| Typical depth (NISQ)| 1 – 3 layers                       | 1 – 6 ansatz blocks               |

Both share a **hybrid loop** but VQE targets *ground states* (eigenvalue problems).

---

## 1) Variational principle  

For any normalised state $|\psi(\boldsymbol\theta)\rangle$,
$$
E(\boldsymbol\theta)=\langle\psi(\boldsymbol\theta)|H|\psi(\boldsymbol\theta)\rangle \;\ge\; E_0,
$$
with equality *iff* $|\psi\rangle$ is the ground state. Therefore minimising $E(\boldsymbol\theta)$ variationally approximates $E_0$.

---

## 2) Molecular Hamiltonians  

### 2.1 Second-quantised form  
$$
H = \sum_{pq} h_{pq} a_p^\dagger a_q \;+\;
     \frac12\sum_{pqrs}h_{pqrs} a_p^\dagger a_q^\dagger a_r a_s.
$$

### 2.2 Qubit mapping  
- **Jordan-Wigner**, **Parity**, or **Bravyi-Kitaev** map fermionic operators to Pauli strings:
$$
H \;=\; \sum_i h_i P_i, \quad P_i\in\{\mathbb{1},X,Y,Z\}^{\otimes n}.
$$

*Example:*
$$
\text{H$_2$} \;\text{(STO-3G)}:
\;
H =  (-1.05)\,Z_0 + 0.39\,Z_1 + 0.18\,Z_0Z_1
     + 0.55\,X_0 X_1 + 0.55\,Y_0 Y_1.
$$

---

## 3) Ansatz families  

| Ansatz | Depth | Qubits | Pros | Cons |
|--------|-------|--------|------|------|
| **UCCSD** (chemistry-inspired) | $O(N^4)$ CNOTs | $4N$ spin-orbitals | High accuracy | Deep on NISQ |
| **Hardware-Efficient** (HEA) | $p(\text{single}+2Q)$ | $n$ | Shallow, topology-friendly | Expressivity limited; barren plateaus |
| **Problem-inspired** (e.g., Heisenberg, QAOA-style) | task-specific | varies | Encodes symmetries | Custom engineering |

*HEA layer illustration (n = 4, depth = 2):*
```
—Ry(θ)——●———Ry——●—
          │        │
—Ry(θ)——●———Ry——●—
          │        │
—Ry(θ)——●———Ry——●—
          │        │
—Ry(θ)——●———Ry——●—
```

---

## 4) Expectation evaluation  

1. **Group commuting Pauli strings** (e.g., diagonalise in same basis) to reduce distinct measurement circuits.  
2. For each group, run circuit → sample counts → compute expectation $\langle P_i\rangle$.  
3. **Energy estimator**  
   $$
   \hat{E}(\boldsymbol\theta)=\sum_i h_i\,\widehat{\langle P_i\rangle}.
   $$
Shot noise scales as $1/\sqrt{M}$; measurement allocation strategies (greedy, variance-aware) cut total shots.

---

## 5) Classical optimisers  

| Optimiser | Shot count / step | Gradient use | Notes |
|-----------|-------------------|--------------|-------|
| **SPSA**  | 2                | stochastic   | Robust to noise, good for ≥20 params |
| **COBYLA**| ~$n+1$          | none         | Faster for small param sets |
| **QN-SR1**| 3–5              | param-shift  | Needs analytic gradients; not always feasible on real HW |

Mitigation of **barren plateaus**: choose shallow ansatz, symmetry-preserving parameters, or layer-wise training.

---

## 6) Hybrid loop pseudo-code  

```python
θ = θ_init
for iter in range(max_iter):
    E = estimate_energy(θ)     # quantum job(s)
    θ = classical_update(θ, E) # e.g., SPSA
    if converged(E): break
```

Real devices: run with error-mitigation (read-out calibration, ZNE) at each iteration or periodically.

---

## 7) Qiskit Nature workflow example (H₂)  

```python
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import SPSA
from qiskit.circuit.library import TwoLocal
from qiskit.primitives import Estimator, Sampler

driver = PySCFDriver(atom='H .0 .0 .0; H .0 .0 0.735', basis='sto3g')
problem = driver.run()

qubit_converter = ParityMapper(num_particles=problem.num_particles)
qubit_op = qubit_converter.map(problem.hamiltonian)

ansatz = TwoLocal(qubit_op.num_qubits, 'ry', 'cx', reps=2)
optimizer = SPSA(maxiter=200)

vqe = VQE(Estimator(), ansatz, optimizer, callback=lambda x: print("E=",x.evaluation))
result = vqe.compute_minimum_eigenvalue(qubit_op)
print("Ground-state energy:", result.eigenvalue.real)
```
On Aer (noise-free) you obtain ≈ −1.137 Ha (chemical accuracy).

---

## 8) Emerging: SQD & SKQD (one-paragraph primer)  

- **SQD** models open-system dynamics with stochastic differential equations; can be used to re-sample noisy expectation values and feed more *robust* gradients to the optimiser.  
- **SKQD** augments SQD with kernel-based regression to interpolate quantum-state trajectories, promising shot reduction and better convergence — active research area.

---

## 9) AI / DS applications  

| Domain | VQE usage |
|--------|-----------|
| **Quantum chemistry** | Drug-discovery screening, catalyst design. Energies feed ML models as labelled data. |
| **Ising optimisation** | Binary feature selection, clustering cost functions. |
| **Quantum kernels** | Ground states of designed Hamiltonians act as feature maps (SKQD can improve noisy kernel estimates). |
| **Generative models** | VQE-like training to learn lowest-free-energy state of Boltzmann machines. |

---

## 10) Mini-exercises (answers in Appendix)

1. Show that measuring Pauli string $X_1 Y_2$ requires basis change $R_X(-\pi/2)$ on qubit 1 and $R_Y(\pi/2)$ on qubit 2 before computational-basis measurement.  
2. Derive two-qubit UCC ansatz for H₂ in STO-3G with single excitation operator $T = \theta(a_0^\dagger a_2 - a_2^\dagger a_0)$ and map it to Pauli rotations.  
3. Implement hardware-efficient ansatz with `TwoLocal` for 4 qubits and compare SPSA vs COBYLA convergence on random 4-qubit Ising Hamiltonian.  
4. Estimate how many shots are required so that statistical error in energy is ≤ 0.001 Ha given operator term variances σ²≈0.25 and 100 commuting groups.  
5. Explain how read-out error mitigation changes the unbiased estimator of $\langle Z\_i\rangle$.

---

## 11) FAQ  

- **“Why UCCSD so deep?”** Each fermionic excitation maps to many CNOTs; Trotterisation plus Jordan-Wigner strings yield long CX chains.  
- **“Can I mix ansatz types?”** Yes—chemistry-inspired excitations plus hardware-efficient layers (adaptive VQE).  
- **“What if I don’t have exact eigenstate count?”** VQE only needs Hamiltonian; number of particles fixed by symmetry sector.  
- **“How to detect convergence?”** Monitor absolute energy change < tolerance over last $k$ iterations, or gradient norm < ε.  

---

## 12) Summary (Session 9)

- **VQE** uses shallow, parameterised circuits + classical optimisers to approximate ground-state energies.  
- Construction pipeline: *Hamiltonian mapping → ansatz → measurement → optimiser*.  
- **UCCSD** gives high accuracy for chemistry; **hardware-efficient** ansätze suit NISQ.  
- Measurement overhead and optimiser noise sensitivity are key bottlenecks; error mitigation and emerging SQD/SKQD techniques help.  
- In AI/DS, VQE underpins quantum-enhanced optimisation and feature-extraction tasks.

---

## 13) Looking ahead  

- **Next Session:** Quantum kernels & QSVMs — leveraging ground-state or variational states as high-dimensional feature maps in machine learning.  
- **Homework 3:**  
  - VQE for H₂ (2-qubit mapping) with SPSA vs COBYLA.  
  - Implement Pauli grouping strategy and compute shot savings.  
  - Brief report on one SQD/SKQD paper (optional bonus).

---

## Appendix — mini-exercise solutions (sketch)

1. Apply $H$ then $S^\dagger$ to measure $Y$; for $X$ use $H$.  
2. Use JW mapping: $a_0^\dagger a_2 - a_2^\dagger a_0 \mapsto \frac{i}{2}(X_0 Z_1 Y_2 - Y_0 Z_1 X_2)$. Exponentiate to $e^{-i\theta(XZY - YZX)/2}$.  
3. SPSA converges within 150 iters; COBYLA within 50 but stalls at higher energy for noisy backend.  
4. Shots per group $M = 4σ²/ε² ≈ 1000$. Total 100 000 shots.  
5. Mitigation inverts calibration matrix $A$: corrected counts $c' = A^{-1} c$; expectation estimator becomes $\langle Z\rangle = (c'_0 - c'_1)/(c'_0+c'_1)$.

